In [1]:
from src.sentinel_api import get_sentinel_api
from src.constant import GEOJSON_PATH, DOWNLOADED_DATA_PATH, REMOVE_ZIPS
import os

# get api 
api = get_sentinel_api()

In [2]:
# search by polygon, time, and Hub query keywords
products_query = api.query(
                    limit = 1,
                    filename = 'S3A_*',
                    producttype = 'OL_2_LFR___')

os.makedirs(DOWNLOADED_DATA_PATH, exist_ok = True) 
products = api.download_all(products_query, DOWNLOADED_DATA_PATH)
print(products)

Downloading: 100%|██████████| 94.8M/94.8M [00:03<00:00, 26.6MB/s]
MD5 checksumming: 100%|██████████| 94.8M/94.8M [00:02<00:00, 34.2MB/s](OrderedDict([('b384df66-ca30-4fce-916a-729fe40ab971', {'id': 'b384df66-ca30-4fce-916a-729fe40ab971', 'title': 'S3A_OL_2_LFR____20200530T132737_20200530T133037_20200531T172050_0179_059_010_1260_LN1_O_NT_002', 'size': 94820603, 'md5': 'D086E41B34D965804B93AC013B33DEE7', 'date': datetime.datetime(2020, 5, 30, 13, 27, 36, 802000), 'footprint': 'POLYGON((129.925 85.7901,121.559 85.7641,113.437 85.6477,105.85 85.4487,99.0488 85.1795,93.0086 84.8483,87.7497 84.4676,83.2021 84.0456,79.2841 83.5948,75.8823 83.1165,72.9482 82.6149,70.3754 82.1001,68.1302 81.5688,66.1313 81.0277,64.3709 80.4774,62.791 79.9211,61.3972 79.3551,60.1311 78.783,58.9885 78.207,57.9471 77.629,68.8741 76.4708,77.892 74.9233,85.1644 73.0939,91.008 71.0644,92.5219 71.4445,94.1022 71.8168,95.7336 72.1732,97.4329 72.5196,99.1967 72.8444,101.03 73.1571,102.927 73.4521,104.883 73.7312,106.902

In [3]:
file_titles = []
zip_files = []

for product in products:
    for product_id in product:
        file_titles.append(product[product_id]['title'])
        zip_files.append(product[product_id]['path'])

        import zipfile
        archive = zipfile.ZipFile(product[product_id]['path'], 'r')
        archive.extractall(path=DOWNLOADED_DATA_PATH)

for zip_file in zip_files:
    if REMOVE_ZIPS:
        os.remove(zip_file)    

In [4]:
from netCDF4 import Dataset
for file_title in file_titles:
    data_path = os.path.join(DOWNLOADED_DATA_PATH, '{}.SEN3'.format(file_title))
    for nc_file_name in sorted(os.listdir(data_path)):
        if nc_file_name.endswith('.nc'):
            nc_file_path =  os.path.join(data_path, nc_file_name)  
            print(nc_file_path)
            rootgrp = Dataset(nc_file_path, "r", format="NETCDF4")

            for i in rootgrp.variables:
                print(i, rootgrp.variables[i].shape)

            print('\n')

./src/data/S3A_OL_2_LFR____20200530T132737_20200530T133037_20200531T172050_0179_059_010_1260_LN1_O_NT_002.SEN3/geo_coordinates.nc
altitude (4091, 4865)
latitude (4091, 4865)
longitude (4091, 4865)


./src/data/S3A_OL_2_LFR____20200530T132737_20200530T133037_20200531T172050_0179_059_010_1260_LN1_O_NT_002.SEN3/instrument_data.nc
FWHM (21, 3700)
detector_index (4091, 4865)
frame_offset (3700,)
lambda0 (21, 3700)
relative_spectral_covariance (21, 21)
solar_flux (21, 3700)


./src/data/S3A_OL_2_LFR____20200530T132737_20200530T133037_20200531T172050_0179_059_010_1260_LN1_O_NT_002.SEN3/iwv.nc
IWV (4091, 4865)
IWV_err (4091, 4865)


./src/data/S3A_OL_2_LFR____20200530T132737_20200530T133037_20200531T172050_0179_059_010_1260_LN1_O_NT_002.SEN3/lqsf.nc
LQSF (4091, 4865)


./src/data/S3A_OL_2_LFR____20200530T132737_20200530T133037_20200531T172050_0179_059_010_1260_LN1_O_NT_002.SEN3/ogvi.nc
OGVI (4091, 4865)
OGVI_err (4091, 4865)


./src/data/S3A_OL_2_LFR____20200530T132737_20200530T133037_20200531